In [1]:
import torch
import torchvision
import torch.nn.functional as F_nn # 神經網路的常用函式
import torchreid # Re-ID 的目標是辨識出不同時間、不同地點拍攝到的同一個人。
from scipy.optimize import linear_sum_assignment  # 匈牙利演算法，將當前幀和上一幀的bbox做比對
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.transforms import functional as F
from torchvision.ops import nms # 處理多重框

# 畫畫
import os
import cv2
import numpy as np
import json
import matplotlib.pyplot as plt
from PIL import Image

import time
import glob # 這很神奇，可以直接在路徑中搜尋要的檔案，不用像我之前那樣慢慢拆再慢慢找
from datetime import datetime
from tqdm import tqdm
from collections import defaultdict
try:
    from pytorch_faster_rcnn_tutorial.faster_RCNN import FasterRCNNLightning 
    # 這個我不確定大家的是否都一樣，有可能不一樣 
except ImportError:
    print("錯誤：導入 FasterRCNNLightning 失敗，去檢查一下當初你的指令是 set 哪裡。")
    exit()

# --- 設定閾值參數區 ---
CONFIDENCE_SCORES = 0.98
#若是用自己的模型就 0.5

TARGET_IDS = [1, 2]
CROP_SIZE = (720, 960) # 特寫影片的解析度 (寬, 高) 模型規定要 288 384，也可用480, 640; 720, 960

NMS_THRESHOLD = 0.2 # 去重框，越小越嚴格
DETECTION_INTERVAL = 1 # 每 3 幀 檢查一遍，節省我寶貴的性能
IOU_THRESHOLD = 0.45 # 追蹤功能(暫定用) 越高越嚴謹(超過代表相配)
# 自己模型就設 0.35

CLASS = ['bg', 'pose'] # 背景是 0，"人臉" 是 1
FEATURE_THRESHOLD = 0.65  # 特徵相似度閾值 越低越寬鬆
# 自己模型就設 0.70

alpha = 0.3 # 平滑數，值越小，舊記憶占比越重

# --- 設定顏色區 ---
BBOX_COLOR = (0, 255, 0) # 綠色
TEXT_COLOR = (0, 0, 0) # 黑色
TEXT_BG_COLOR = (255, 255, 255) # 白底
# 所以會是白底黑字 綠框

ID_COLORS = [ # ID 的顏色
    (255, 0, 0),      # 紅色
    (0, 0, 255),      # 藍色
    (0, 255, 255),    # 青色
    (255, 128, 0),    # 橘色
    (255, 0, 255),    # 紫紅
    (128, 0, 255),    # 紫色
    (255, 0, 128),    # 桃紅
    (128, 64, 0),     # 棕色
]

# --- 追蹤 --
class DeepSORT:
    def __init__(self, max_age=35, min_hits=1, feature_threshold=FEATURE_THRESHOLD): # 
        # min_hits 越小，ID 確認越快
        # max_age 越大，ID 越不容易斷
        self.max_age = max_age  # 最大消失幀數
        self.min_hits = min_hits  # 最少偵測次數才顯示
        self.feature_threshold = feature_threshold
        self.tracks = {} # 儲存所有的資料，以 id為鍵
        self.next_id = 1

        # 載入預訓練模型，專門用於從影像中提取特徵向量，來區分不同的人。
        self.feature_extractor = torchreid.models.build_model(
            name='osnet_x0_25',
            num_classes=1, # 對特徵提取不重要
            loss='softmax', # 也跳過
            pretrained=False # 沒有要重訓練
        )
        model_name = self.feature_extractor.__class__.__name__ # class 返回物件類別，name = osnet_x0_25(專門抓人類)
        print(f"正在載入 Re-ID 模型 ({model_name})...")
        
        # 載入預訓練權重
        model_weights_path = r'D:\rcnn\PyTorch-Object-Detection-Faster-RCNN-Tutorial\weights\osnet_x0_25_market_256x128_amsgrad_ep180_stp80_lr0.003_b128_fb10_softmax_labelsmooth_flip.pth'
        
        torchreid.utils.load_pretrained_weights(self.feature_extractor, model_weights_path)
        # 3. 設定為評估模式並移至 GPU
        self.feature_extractor.eval()
        if torch.cuda.is_available():
            self.feature_extractor = self.feature_extractor.cuda()
        print("Re-ID 模型載入成功！")
        
        
    
    def extract_features(self, image_rgb, boxes):
        """提取特徵"""
        if len(boxes) == 0:
            return []
        
        features = [] # 要放特徵向量
        with torch.no_grad():
            # 這意思是禁用梯度計算，因為我現在只要跑而已，沒有要像訓練那樣重新更新裡面的權重或是資源
            for box in boxes:
                x1, y1, x2, y2 = box.astype(int)
                
                # 確保座標在圖像範圍內
                x1 = max(0, x1)
                y1 = max(0, y1)
                # image_rgb = (高度, 寬度, rgb三種顏色通道)
                x2 = min(image_rgb.shape[1], x2)
                y2 = min(image_rgb.shape[0], y2)
                
                temp = image_rgb[y1:y2, x1:x2] # 這個 temp 會是 y1-y2, x1-x2 的大小範圍 
                # temp = 高、寬
                
                if temp.size == 0 or temp.shape[0] < 10 or temp.shape[1] < 10:
                    features.append(None)
                    continue
                
                # 調整大小到 Re-ID 模型要的固定尺寸
                temp = cv2.resize(temp, (128, 256))
                
                # 把 numpy 轉 tensor
                face_tensor = F.to_tensor(temp)
                # 正規化
                face_tensor = F.normalize(face_tensor, 
                                            # 平均值
                                            mean=[0.485, 0.456, 0.406],
                                              #   紅      綠      藍
                                            # 標準差
                                            std=[0.229, 0.224, 0.225])
                
                face_tensor = face_tensor.unsqueeze(0) 
                # 就是在 0 位置 append 一個 1 代表批次大小(一次處理幾個啦)
                
                if torch.cuda.is_available():
                    face_tensor = face_tensor.cuda()
                
                # 輸出特徵向量
                # self.feature_extractor 是一個物件，代表我們剛剛建立的 RE-ID 模型
                feature = self.feature_extractor(face_tensor) # 把tesor 送到 RE-ID 模型
                feature = feature.squeeze().cpu().numpy()
                # 正規化特徵向量
                feature = feature / (np.linalg.norm(feature) + 1e-6)
                features.append(feature)
        
        return features
    
    def cosine_similarity(self, feat1, feat2):
        """計算餘弦相似度"""
        if feat1 is None or feat2 is None:
            return 0.0
        return np.dot(feat1, feat2)  # 已經正規化過了
    
    def update(self, boxes, scores, image_rgb=None):
        """更新追蹤（需要 RGB 圖像來提取特徵）"""
        # 如果沒有圖像，降級為簡單 IOU 追蹤
        if image_rgb is None:
            return self.simple_update(boxes, scores)
        
        # 提取特徵
        features = self.extract_features(image_rgb, boxes)
        
        # 計算成本矩陣
        track_ids = list(self.tracks.keys())
        n_tracks = len(track_ids)
        n_detections = len(boxes) # 所有偵測到的物件的邊界框座標
        
        if n_tracks > 0 and n_detections > 0: # 舊的有，也有新的
            # 建立成本矩陣
            cost_matrix = np.ones((n_tracks, n_detections)) * 1e6
            #                      定義矩陣大小
            
            # cost_matrix[i, j] 代表第 i 個追蹤物件與第 j 個偵測框之間的不相似度。
            #                     或者應該說 舊的 第i個物件和 新的 第 j 個物件的不相似度
            for i, track_id in enumerate(track_ids):
                track = self.tracks[track_id]
                for j, (box, feature) in enumerate(zip(boxes, features)): # 先包起來然後 給每個東西作編號首引
                    # box :邊界框座標， feature: 對應的特徵向量。
                    # 特徵相似度
                    if 'feature' in track and feature is not None: # 先確認 feature 這個東西是存在的
                        feat_sim = self.cosine_similarity(track['feature'], feature)
                        #                                  過去的特徵      這幀偵測到的特徵
                    else:
                        feat_sim = 0
                    
                    # IOU
                    iou = self.calculate_iou(track['box'], box) # 然後換 IOU
                    
                    # 組合成本（特徵為主，IOU 為輔）
                    if feat_sim > self.feature_threshold or iou > IOU_THRESHOLD:
                        cost = (1 - feat_sim) * 0.7 + (1 - iou) * 0.3
                        cost_matrix[i, j] = cost
                        # cost_matrix[i, j] 代表第 i 個追蹤物件與第 j 個偵測框之間的相似度。
            
            # 匈牙利算法
            row_indices, col_indices = linear_sum_assignment(cost_matrix) # 輸出好像是兩個列表，代表各自對應的最小成本(最相似)
            """ 我搞懂匈牙利在幹嘛了 他舊是單純把所有「舊匹配」和 「新的匹配」之間的差額，取最小
            而 row 後面會是 舊，col 會是新 按照順序排 像是 zip一樣互相對應。"""
            
            matched_tracks = set()
            matched_detections = set()
            
            # 處理匹配
            for row, col in zip(row_indices, col_indices):
                # 新舊物件的差異，越小越像
                if cost_matrix[row, col] < 1:  # 成本閾值
                    track_id = track_ids[row]
                    self.tracks[track_id]['box'] = boxes[col]
                    self.tracks[track_id]['score'] = scores[col]
                    self.tracks[track_id]['lost'] = 0
                    self.tracks[track_id]['hits'] += 1
                    
                    # 更新特徵
                    if features[col] is not None:
                        self.tracks[track_id]['feature'] = (1 - alpha) * self.tracks[track_id]['feature'] + alpha * features[col]
                        #                                                  舊特徵                              新特徵
                    
                    matched_tracks.add(track_id)
                    # 記錄當前幀，所有匹配到新偵測到的舊追蹤物件的 ID
                    matched_detections.add(col)
            
            # 先找舊追蹤
            for track_id in track_ids:
                # 若是沒有在匹配裡面
                if track_id not in matched_tracks:
                    self.tracks[track_id]['lost'] += 1
                    if self.tracks[track_id]['lost'] > self.max_age:
                        del self.tracks[track_id]
            
            # 未匹配的偵測（新追蹤）
            for j in range(n_detections):
                if j not in matched_detections:
                    self.tracks[self.next_id] = {
                        'box': boxes[j],
                        'score': scores[j],
                        'lost': 0,
                        'hits': 1,
                        'id': self.next_id,
                        'feature': features[j] if j < len(features) else None
                    }
                    self.next_id += 1
        
        elif n_detections > 0:
            # 全部都是新追蹤
            for j, box in enumerate(boxes):
                self.tracks[self.next_id] = {
                    'box': box,
                    'score': scores[j],
                    'lost': 0,
                    'hits': 1,
                    'id': self.next_id,
                    'feature': features[j] if j < len(features) else None
                }
                self.next_id += 1
        
        else:
            # 沒有偵測
            for track_id in list(self.tracks.keys()):
                self.tracks[track_id]['lost'] += 1
                if self.tracks[track_id]['lost'] > self.max_age:
                    del self.tracks[track_id]
        
        # 只返回穩定的追蹤
        stable_tracks = {}
        for track_id, track in self.tracks.items():
            if track['hits'] >= self.min_hits:
                stable_tracks[track_id] = track
        
        return stable_tracks
    
    def simple_update(self, boxes, scores):
        """簡單更新（沒有圖像時的備用方案）"""
        if len(boxes) == 0:
            for track_id in list(self.tracks.keys()):
                self.tracks[track_id]['lost'] += 1
                if self.tracks[track_id]['lost'] > self.max_age:
                    del self.tracks[track_id]
            return self.tracks
        
        # ... 簡化的 IOU 匹配邏輯 ...
        return self.tracks
    
    def calculate_iou(self, box1, box2):
        """計算 IOU"""
        x1 = max(box1[0], box2[0])
        y1 = max(box1[1], box2[1])
        x2 = min(box1[2], box2[2])
        y2 = min(box1[3], box2[3])
        
        if x2 < x1 or y2 < y1:
            return 0.0
        
        intersection = (x2 - x1) * (y2 - y1)
        area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
        area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
        union = area1 + area2 - intersection
        
        if union == 0:
            return 0.0
        return intersection / union

"""    
https://chih-sheng-huang821.medium.com/%E6%A9%9F%E5%99%A8-%E6%B7%B1%E5%BA%A6%E5%AD%B8%E7%BF%92-%E7%89%A9%E4%BB%B6%E5%81%B5%E6%B8%AC-non-maximum-suppression-nms-aa70c45adffa
這篇有講怎麼去重複的框框，重點整理:
    1. 先看哪個BBox的信心程度最高，那個BBox會進去「確定是物件集合」(途中的selected objects)內
    2. 其他BBox和剛選出來的BBox算IoU，然後算出來的IoU大於設定好的閾值的BBox，那些BBox的信心度會被設定為0(也就是這個BBox重複計算要刪掉)。"""
# 自信度由大到小(左->又)排，看有幾個物件就 run 幾次去計算各自的 IOU 是否 > 閾值(我的理解是這樣)

def apply_nms(boxes, scores, labels, nms_threshold):
    if(len(boxes) ==0):
        return boxes, scores, labels

    # 將numpy array轉為tensor
    boxes_tensor = torch.from_numpy(boxes).float()
    scores_tensor = torch.from_numpy(scores).float()
    
    # 應用 NMS
    keep_indices = nms(boxes_tensor, scores_tensor, nms_threshold)

    # 轉回numpy並返回過濾後的結果
    keep_indices = keep_indices.numpy()
    return boxes[keep_indices], scores[keep_indices], labels[keep_indices]
    
def load_model(checkpoint_path, device):
    print(f"正在從 {checkpoint_path} 載入模型")
    model = FasterRCNNLightning.load_from_checkpoint(
        checkpoint_path=checkpoint_path, # .ckpt 檔案的路徑
        map_location=device
    )
    model.eval() # 把 dropout 功能關掉，只剩下評估模式，
                 # 這樣不會汙染到我們的訓練模型。非常重要的機器學習觀念 !!
    model.to(device)
    print("模型載入成功並已設定為評估模式。")
    return model

def working(input_path, output_path, model, device, model_type):
    print(f"---> 正在處理影片 {os.path.basename(input_path)}")

    cap = cv2.VideoCapture(input_path) # opencv 會自己去解碼原影片
    if not cap.isOpened():
        print(f" !! {os.path.basename(input_path)} 打不開，快去檢查")
        return
    frame_w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) # 取寬
    frame_h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) # 取長
    # w, h 和原影片一樣
    fps = cap.get(cv2.CAP_PROP_FPS)
    if fps == 0: 
        fps = 30.0

    # 設定影片編碼器與建立 VideoWriter 物件
    fourcc = cv2.VideoWriter_fourcc(*'mp4v') # 使用 mp4v 編碼
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_w, frame_h))
    #                        輸出路徑、檔案格式、幀率  和  畫面大小。

    base, form = os.path.splitext(output_path)
    output_id1 = f"{base}-1{form}"
    output_id2 = f"{base}-2{form}"

    json_id1 = f"{base}-1"
    json_id2 = f"{base}-2"
    os.makedirs(json_id1, exist_ok=True)
    os.makedirs(json_id2, exist_ok=True)

    out_id1 = cv2.VideoWriter(output_id1, fourcc, fps, CROP_SIZE)
    out_id2 = cv2.VideoWriter(output_id2, fourcc, fps, CROP_SIZE) # 用論文規定的(推薦)的 size
    black_frame = np.zeros((CROP_SIZE[1], CROP_SIZE[0], 3), dtype=np.uint8) # 如果這個 id1 id2 不見了，就用黑幕代替
    # np.zeros (行, 列, 維度) 在我們這個例子應該是 高 寬
    

    frame_count = 0
    start_time = datetime.now()
    tracker = DeepSORT(max_age=35, min_hits=1, feature_threshold=FEATURE_THRESHOLD)
    tracked_objects = {}
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break # 結束

        clean_frame = frame.copy() # 複製一份乾淨的幀
        frame_count += 1
        
        # 新增條件，我要每幾幀再檢查，減少消耗
        if frame_count % DETECTION_INTERVAL== 0 or frame_count == 1:
            
            # 將 OpenCV 的 BGR 格式轉為 RGB 格式
            image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            # 轉為 PyTorch Tensor，並將像素值從 [0, 255] 縮放到 [0.0, 1.0]
            image_tensor = F.to_tensor(image_rgb)
            # 增加一個批次維度 (batch dimension)，並移動到指定設備
            image_tensor = image_tensor.unsqueeze(0).to(device)
            # Tensor 格式通常是 (顏色通道, 高度, 寬度)，但是深度學習前面會有一個代表 批次
            # 在第零維度插入新維度 變成 [1, x, y, z]， 1就是批次大小，代表一次只處理一張影像
            
    
            with torch.no_grad(): # 就跟 eval 一樣，測試階段記得加
                predictions = model(image_tensor) # 呼叫我下載的模型
                
            # predictions[0] 包含 'boxes', 'labels', 'scores'
            boxes = predictions[0]['boxes'].data.cpu().numpy()
            scores = predictions[0]['scores'].data.cpu().numpy()
            labels = predictions[0]['labels'].data.cpu().numpy()

            if model_type == "coco":
                mask = (scores >= CONFIDENCE_SCORES) & (labels == 1) # 我們只抓人類就好
            else:
                mask = scores >= CONFIDENCE_SCORES
            # 過濾掉低於信心度閾值的結果
            # 布林函式 true就加進來，false就掰掰
            detected_boxes = boxes[mask].astype(np.int32)
            detected_labels = labels[mask]
            detected_scores = scores[mask]

            # 用 nms 去重
            detected_boxes, detected_scores, detected_labels = apply_nms(
                detected_boxes, detected_scores, detected_labels, NMS_THRESHOLD
            )

            # 更新追蹤器
            tracked_objects = tracker.update(detected_boxes, detected_scores, image_rgb)
            
        else:
            tracked_objects = tracker.update(np.array([]), np.array([]), None)
            

        did_ids = set() # 做過的
        for track_id, track_info in tracked_objects.items():
            box = track_info['box'].astype(np.int32)
            score = track_info['score']
            id_color = ID_COLORS[ track_id % len(ID_COLORS)]
            
            # 繪製偵測框
            cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), id_color, 8)
            #                   左上角座標 (x1, y1)、右下角座標 (x2, y2)、顏色（綠色）、線條粗細（2 像素）

            box_height = box[3] - box[1]  # y2-y1
            font_scale = min(2.6, max(0.5, box_height / 200))
            thin = max(1, int(font_scale * 2))
            
            # 準備要顯示的文字
            if model_type == "coco":
                text = f"ID{track_id} person: {score:.2f}"
            else:
                text = f"ID{track_id} face: {score:.2f}"
            
            # 計算文字大小以便繪製背景
            (text_width, text_height), baseline = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, font_scale, 2)
            #                                                     文字內容、字型、縮放係數 0.8 和 線條粗細 4
            
            # 繪製文字背景
            cv2.rectangle(frame, (box[0], box[1] - text_height - baseline), (box[0] + text_width, box[1]), TEXT_BG_COLOR, -1)
            # 單位就一樣， -1 代表填滿
            
            # 繪製文字
            cv2.putText(frame, text, (box[0], box[1] - baseline), cv2.FONT_HERSHEY_SIMPLEX, font_scale, TEXT_COLOR, thin)
            #          目標影像、文字內容、左下角座標、                       字型、         縮放係數、顏色（黑色）和線條粗細

            if track_id in TARGET_IDS:
                did_ids.add(track_id)
                
                # 計算 BBox 中心點和最大邊長 (為了轉成正方形)
                x1, y1, x2, y2 = box
                center_x = (x1 + x2) // 2
                center_y = (y1 + y2) // 2
                width = x2 - x1
                height = y2 - y1
                max_way = max(width, height)
                max_way = int(max_way * 1.2) # 多一點大小
                
                # 計算正方形的新座標
                new_x1 = center_x - max_way // 2 # max_way // 2  ==>  正方形的半邊長
                new_y1 = center_y - max_way // 2
                new_x2 = new_x1 + max_way
                new_y2 = new_y1 + max_way

                # 確保裁切範圍不超出原始畫面
                new_x1 = max(0, new_x1)
                new_y1 = max(0, new_y1)
                new_x2 = min(frame_w, new_x2)
                new_y2 = min(frame_h, new_y2)

                # 從原始幀中裁切出專屬於個人的圖像
                new_personal_bbox_with_box = frame[new_y1:new_y2, new_x1:new_x2] # 用有框的(用再特寫影片和之後 mp )
                new_personal_bbox_clean = clean_frame[new_y1:new_y2, new_x1:new_x2] # 先準備給之後的

                # 5. 檢查裁切是否成功，再縮放並寫入對應影片
                if new_personal_bbox_with_box.size > 0:
                    resized = cv2.resize(new_personal_bbox_with_box, CROP_SIZE)
                    if track_id == 1:
                        out_id1.write(resized)
                    elif track_id == 2:
                        out_id2.write(resized)

                    if frame_count % 3 == 0: # 一樣每三幀取一次
                        # ori_x1, ori_y1 = 原始偵測框的左上角座標
                        # new_x1, new_y1 = 擴大後的正方形的左上角座標
                        ori_x1, ori_y1, ori_x2, ori_y2 = box
                        ori_w = ori_x2 - ori_x1 
                        ori_h = ori_y2 - ori_y1
    
                        relative_x = ori_x1 - new_x1
                        relative_y = ori_y1 - new_y1
    
                        crop_h, crop_w, _ = new_personal_bbox_with_box.shape
    
                        target_w, target_h = CROP_SIZE
                        scale_w = target_w / crop_w
                        scale_h = target_h / crop_h
    
                        final_bbox = [
                            int(relative_x * scale_w),
                            int(relative_y * scale_h),
                            int(ori_w * scale_w),
                            int(ori_h * scale_h)
                        ]
                        
                        json_data = {
                            "width": target_w,
                            "height": target_h,
                            "bbox": final_bbox
                        }
    
                        json_filename = f"frame_{frame_count:012d}.json"
                        if track_id == 1:
                            json_path = os.path.join(json_id1, json_filename)
                        else: 
                            json_path = os.path.join(json_id2, json_filename)

                        with open(json_path, 'w') as f:
                            json.dump(json_data, f, indent=4)

                        
                        if new_personal_bbox_clean.size > 0:
                            resized_clean = cv2.resize(new_personal_bbox_clean, CROP_SIZE)
                            image_filename = f"frame_{frame_count:012d}.jpg"
                            if track_id == 1:
                                image_path = os.path.join(json_id1, image_filename)
                            else:
                                image_path = os.path.join(json_id2, image_filename)
                            cv2.imwrite(image_path, resized_clean)
                    
        # 6. 寫入畫格
        out.write(frame)
        # 如果有意外，id1、id2 突然斷掉
        if 1 not in did_ids:
            out_id1.write(black_frame)
        if 2 not in did_ids:
            out_id2.write(black_frame)

    # 7. 資源釋放
    print(f"--> 影片處理完成: {os.path.basename(output_path)}")
    print(f"--> 特寫影片已儲存: {os.path.basename(output_id1)}")
    print(f"--> 特寫影片已儲存: {os.path.basename(output_id2)}")
    cap.release()
    out.release()
    out_id1.release()
    out_id2.release()
        
    

def main():

    # 選項1: "mine"  -> 我自己訓練的 version_5 
    # 選項2: "coco" -> 用別人訓練好的 coco
    # MODEL_TO_USE = "mine"
    MODEL_TO_USE = "coco"
    
    # 先確認是用GPU 免得像我一樣用成cpu
    device =  torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"現在正在使用 {device} 跑模型")
    
    project_dir = r'D:\rcnn\PyTorch-Object-Detection-Faster-RCNN-Tutorial'
    input_dir =  os.path.join(project_dir, 'test_videos')
    output_dir = os.path.join(project_dir, 'output_videos')
    
    if not os.path.exists(input_dir):
        os.makedirs(input_dir)
        print("已自動建立輸入資料夾")
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print("已自動建立輸出資料夾")

    model = None
    if MODEL_TO_USE == "mine":
        print("正在使用自己的 version_5 模型")
        try:
            model_checkpoints_dir = r"D:\rcnn\PyTorch-Object-Detection-Faster-RCNN-Tutorial\lightning_logs\version_5\checkpoints\epoch=8-step=54.ckpt"
            model = load_model(model_checkpoints_dir, device)
            # 這邊我是用 version_5 ，你們可以依據自己的模型權重版本去調整。
            print("Faster r-cnn 自己訓練的模型準備好了!")
        except Exception as e:
            print(f"載入自己的模型時發生錯誤: {str(e)}")
            print(" 可能發生路徑錯誤或檔案名稱錯誤等等資訊")
            return
            
    elif MODEL_TO_USE == "coco":
        print("用 coco 訓練好的模型")
        model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights=torchvision.models.detection.FasterRCNN_ResNet50_FPN_Weights.DEFAULT)
        model.eval()
        model.to(device)
        print("coco 模型準備好了")
        
    #找出所有影片
    video_list = glob.glob(os.path.join(input_dir, '*.mp4'))
    if not video_list:
        print("還沒找到任何 mp4 輸入檔")
        return
        
    print(f"找到 {len(video_list)}個 mp4檔案")
    #tracker = DeepSORT(feature_threshold=FEATURE_THRESHOLD)
    for v in tqdm(video_list, desc="處理中"):
        input_path = v
        output_path = os.path.join(output_dir, os.path.basename(v))
        # 輸入輸出檔案名稱要一樣。
        if(os.path.exists(output_path)):
            print(f"已有 {v} ，所以跳過")
            continue
        working(input_path, output_path, model, device, MODEL_TO_USE)
        
    print("所有影片皆已完成")


main()

C:\Users\LOUIS\anaconda3\envs\rcnn-v2\lib\site-packages\torchreid\reid\metrics\rank.py:11: UserWarning: Cython evaluation (very fast so highly recommended) is unavailable, now use python evaluation.
  warnings.warn(


現在正在使用 cuda 跑模型
用 coco 訓練好的模型
coco 模型準備好了
找到 11個 mp4檔案


處理中:   0%|                                                                                                                       | 0/11 [00:00<?, ?it/s]

---> 正在處理影片 test001.mp4
正在載入 Re-ID 模型 (OSNet)...
Successfully loaded pretrained weights from "D:\rcnn\PyTorch-Object-Detection-Faster-RCNN-Tutorial\weights\osnet_x0_25_market_256x128_amsgrad_ep180_stp80_lr0.003_b128_fb10_softmax_labelsmooth_flip.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']
Re-ID 模型載入成功！


C:\Users\LOUIS\anaconda3\envs\rcnn-v2\lib\site-packages\torchreid\reid\utils\torchtools.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fpath, m

...

--> 影片處理完成: test009.mp4
--> 特寫影片已儲存: test009-1.mp4
--> 特寫影片已儲存: test009-2.mp4
---> 正在處理影片 test010.mp4
正在載入 Re-ID 模型 (OSNet)...
Successfully loaded pretrained weights from "D:\rcnn\PyTorch-Object-Detection-Faster-RCNN-Tutorial\weights\osnet_x0_25_market_256x128_amsgrad_ep180_stp80_lr0.003_b128_fb10_softmax_labelsmooth_flip.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']
Re-ID 模型載入成功！


處理中:  91%|████████████████████████████████████████████████████████████████████████████████████████████████████          | 10/11 [20:06<01:34, 94.15s/it]

--> 影片處理完成: test010.mp4
--> 特寫影片已儲存: test010-1.mp4
--> 特寫影片已儲存: test010-2.mp4
---> 正在處理影片 test011.mp4
正在載入 Re-ID 模型 (OSNet)...
Successfully loaded pretrained weights from "D:\rcnn\PyTorch-Object-Detection-Faster-RCNN-Tutorial\weights\osnet_x0_25_market_256x128_amsgrad_ep180_stp80_lr0.003_b128_fb10_softmax_labelsmooth_flip.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']
Re-ID 模型載入成功！


處理中: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [20:34<00:00, 112.25s/it]

--> 影片處理完成: test011.mp4
--> 特寫影片已儲存: test011-1.mp4
--> 特寫影片已儲存: test011-2.mp4
所有影片皆已完成
